## Imports

In [43]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import time

## Data Read-in

In [69]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

2250


In [3]:
gdf = gpd.read_file('Palm_Beach_County_Boundary.geojson')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input County Name:')
county_name = input()

Input County Name:
Palm Beach County


In [5]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
613


In [6]:
print('Input total sales volume ($$$ million as 100_000_000) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million as 100_000_000) from previous (not last) month:
301_000_000


In [7]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
265000


In [8]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
238


## Data Clean

In [70]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [71]:
df = df.dropna(subset=['SOLD DATE'])

In [72]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: August ---- Month before report month: July


In [73]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [74]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    566
Name: count, dtype: int64
-------
$/SQUARE FEET
False    564
True       2
Name: count, dtype: int64
-------
YEAR BUILT
False    566
Name: count, dtype: int64
-------


In [75]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-7102-H/home/191915656


In [76]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [77]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [78]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [79]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
519,PAST SALE,August-30-2024,Condo/Co-op,1601 S Flagler Dr Unit 302o,West Palm Beach,FL,33401.0,31000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1601...,Beaches MLS,RX-10990917,N,Y,26.699285,-80.051359
524,PAST SALE,August-6-2024,Condo/Co-op,161 Preston D,Boca Raton,FL,33434.0,60000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/161-Prest...,Beaches MLS,RX-10980774,N,Y,26.393597,-80.178271
527,PAST SALE,August-19-2024,Condo/Co-op,515 Monaco K,Delray Beach,FL,33446.0,60000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/515-Mon...,Beaches MLS,RX-10981233,N,Y,26.450034,-80.150653
491,PAST SALE,August-8-2024,Condo/Co-op,2811 S Garden Dr #101,Lake Worth,FL,33461.0,60000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2811-Gard...,Beaches MLS,RX-10993542,N,Y,26.633622,-80.085302
522,PAST SALE,August-14-2024,Condo/Co-op,183 Norwich H,West Palm Beach,FL,33417.0,60000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/183-...,Beaches MLS,RX-10982299,N,Y,26.720040,-80.120173
529,PAST SALE,August-30-2024,Condo/Co-op,319 Norwich N,West Palm Beach,FL,33417.0,60000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/319-...,Beaches MLS,RX-10983090,N,Y,26.719944,-80.120176
509,PAST SALE,August-30-2024,Condo/Co-op,575 Monaco L,Delray Beach,FL,33446.0,65000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/575-Mon...,Beaches MLS,RX-10991156,N,Y,26.450034,-80.150653
530,PAST SALE,August-15-2024,Condo/Co-op,527 Brittany K,Delray Beach,FL,33446.0,65000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/527-Bri...,Beaches MLS,RX-10967656,N,Y,26.444317,-80.153314
540,PAST SALE,August-30-2024,Condo/Co-op,2770 S Garden Dr #201,Lake Worth,FL,33461.0,66500.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/2770-Gard...,Beaches MLS,RX-10946288,N,Y,26.632569,-80.084665
499,PAST SALE,August-26-2024,Condo/Co-op,670 N Monaco N #670,Delray Beach,FL,33446.0,67000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/670-Mon...,MARMLS,A11598245,N,Y,26.441373,-80.149819


In [80]:
print(df_filtered['URL'].iloc[529])

https://www.redfin.com/FL/West-Palm-Beach/319-Norwich-N-33417/home/42146180


In [25]:
# df_filtered = df_filtered.drop(index=556)

In [81]:
# Correct the prices, if needed
df_filtered.at[519,'PRICE']=(310_000)

# Correct the psf, if needed
df_filtered.at[519,'$/SQUARE FEET']=(310_000/673)

In [22]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [23]:
# print(df_filtered['URL'].iloc[291])

In [82]:
df_filtered.reset_index(inplace=True)

In [83]:
df_filtered = df_filtered.drop(columns='index')

In [84]:
df_filtered.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [85]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
491,60000.0,2811 S Garden Dr #101,Lake Worth,67.0
540,66500.0,2770 S Garden Dr #201,Lake Worth,74.0
490,72000.0,2855 Garden Dr #202,NaN,81.0
500,71128.0,273 Monaco Unit F,Delray Beach,81.0
527,60000.0,515 Monaco K,Delray Beach,83.0
389,140000.0,11 Stratford Dr E Unit H,Boynton Beach,86.0
522,60000.0,183 Norwich H,West Palm Beach,88.0
321,125000.0,1303 Bridgewood Dr,Boca Raton,89.0
509,65000.0,575 Monaco L,Delray Beach,90.0
390,170000.0,19179 Sabal Lake Dr #5154,Boca Raton,90.0


In [25]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [86]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [87]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [88]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [89]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [90]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [91]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip='Palm Beach County',name='Palm Beach County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [92]:
m.save('index.html')

## Summary Info

In [98]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [99]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS'],
      dtype='object')

In [100]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [101]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Ocean-Ridge/5700-Old-Ocean-Blvd-33435/unit-R/home/42432027


## Map URL Snagger

In [102]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_condo_sales_month_ending_august_2024


## Get Summary Data

In [103]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 546
--------
Total sale price: $233,406,664
--------
Median sale price: $260,000
--------
Max sale price: $5,800,000
--------
Min sale price: $60,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,130
--------
Min price per square foot: $67
--------
Median price per square foot: $226
------------------------------------------------
CONDO AGES
Newest building: 2024.0
----------
Oldest building: 1957.0
----------
Average building age: 1983.6776556776556


## Collect Agent Info

In [104]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [121]:
df_filtered.reset_index(inplace=True,drop=True)

In [122]:
max_price_index = df_filtered['PRICE'].idxmax()
min_price_index = df_filtered['PRICE'].idxmin()

max_price_psf_index = df_filtered['$/SQUARE FEET'].idxmax()
min_price_psf_index = df_filtered['$/SQUARE FEET'].idxmin()

max_year_built_index = df_filtered['YEAR BUILT'].idxmax()
min_year_built_index = df_filtered['YEAR BUILT'].idxmin()

In [123]:
index_list = [max_price_index,min_price_index,
             max_price_psf_index,min_price_psf_index,
             max_year_built_index,min_year_built_index]

In [137]:
response_list = []

for index, row in df_filtered.iterrows():
    if index in index_list:
        response_list.append(process_response(row['URL']))
        
response_df = pd.DataFrame(response_list)

merged_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url',how='outer')

In [138]:
merged_df['list_agent_name_1'].unique()

array([nan, 'Julia Shust', "Michael D'Addio", 'Refik Kalyoncu',
       'Sharon Siano', 'Carol Sollak', 'Kevin Spina'], dtype=object)

In [140]:
print(merged_df.iloc[481]['url'])



https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-2201/home/187292011


In [141]:
print(merged_df['URL'].iloc[0])



https://www.redfin.com/FL/Boca-Raton/1-N-Ocean-Blvd-33432/unit-304/home/42625336


In [142]:
merged_df[merged_df['list_agent_name_1'] == 'Carol Sollak']

,SOLD DATE,ADDRESS,CITY,PRICE,BEDS,BATHS,LOCATION,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,...,FULL_ADDRESS,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
481,August-6-2024,200 Arkona Ct #2201,West Palm Beach,5800000.0,3.0,3.5,La Clara,2023.0,NaN,1837.0,...,200 Arkona Ct #2201 West Palm Beach,https://www.redfin.com/FL/West-Palm-Beach/200-...,Carol Sollak,Engel & Volkers Wellington,None,None,Carol Sollak,Engel & Volkers Wellington,None,None


In [143]:
df_filtered = merged_df

In [146]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")

Most Expensive
La Clara, 200 Arkona Ct #2201 West Palm Beach | Price $5,800,000 | $1,837 psf | Year built: 2023 | Listing agents: Carol Sollak with Engel & Volkers Wellington and None with None | Buyer's agents: Carol Sollak with Engel & Volkers Wellington and None with None
Least Expensive
Century Village Condo, 161 Preston D Boca Raton | Price $60,000 | $100 psf | Year built: 1981 | Listing agents: nan with nan and nan with nan | Buyer's agents: nan with nan and nan with nan
Highest Price Per Square Foot
Marsh Harbour 50 Condo, 1940 Freeport Ln Riviera Beach | Price $1,821,000 | $2,130 psf | Year built: 2006 | Listing agents: Refik Kalyoncu with Lara Real Estate Group and None with None | Buyer's agents: Brenda White with Atlantic Florida Properties Inc and None with None
Lowest Price Per Square Foot
Lake Clarke Gardens Condo 25, 2811 S Garden Dr #101 Lake Worth | Price $60,000 | $67 psf | Year built: 1971 | Listing agents: Julia Shust with Robert Slack LLC and None with None | Buyer

## Get condo names and locations for most and least expensive

In [147]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

In [148]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    second_nut_graf_ref = 'up'
    social = 'higher'
    seo = 'increased'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    second_nut_graf_ref = 'down'
    social = 'lower'
    seo = 'dropped'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    second_nut_graf_ref = 'on par with'
    social = 'equal'
    seo = "did't change"
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rose'
elif prior_month_closings > subject_month_closings:
    dek_rf ='fell'
else:
    dek_rf = 'equaled'

In [151]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS', 'url',
       'list_agent_name_1', 'list_broker_name_1', 'list_agent_name_2',
       'list_broker_name_2', 'buy_agent_name_1', 'buy_broker_name_1',
       'buy_agent_name_2', 'buy_broker_name_2'],
      dtype='object')

In [163]:
# Get values for the top sale (max price)
top_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_1']
top_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_1']
top_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_2']
top_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_2']
top_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_1']
top_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_1']
top_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_2']
top_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_2']

# Get values for the bottom sale (min price)
bottom_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_1']
bottom_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_1']
bottom_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_2']
bottom_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_2']
bottom_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_1']
bottom_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_1']
bottom_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_2']
bottom_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_2']

# Now create the formatted string
test_string = f'''


TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

'''

print(test_string)





TOP SALE LISTING AGENTS:

Carol Sollak with Engel & Volkers Wellington and None with None

TOP SALE BUYER AGENTS:

Carol Sollak with Engel & Volkers Wellington and None with None

BOTTOM SALE LISTING AGENTS:

nan with nan and nan with nan

BOTTOM SALE BUYER AGENTS:

nan with nan and nan with nan




In [161]:
df_filtered["PRICE"].idxmax()

481

In [165]:
story_string = f'''
\033[1mHED:\033[0m {county_name} condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county_name} {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m {county_name}’s condo sales volume {seo} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m {county_name} had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county_name}, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m

{county_name}’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {second_nut_graf_ref} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max():,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max():,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. Unit XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [166]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")


HED: Palm Beach County condo sales dollar volume drops in August to $233M 
DEK: Number of closings fell to 546 from 613 in July
FEATURED HED:
SEO HED: Palm Beach County August Condo Sales Report 
SEO DESCRIPTION: Palm Beach County’s condo sales volume dropped to $233 million in August.
AUTHOR: Adam Farence
RESEARCH: 
Social: Palm Beach County had $233 million in condo sales in August, lower than $301 million from July, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, monthly condo sales, condos, condo market, La Clara, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:

Palm Beach County’s NEWS PEG HERE.

August condo sales totaled $233 million, falling from $301 million in July. Brokers closed 546 sales last month, down from 613 sales in July and XXX in XXX, Multiple Listing Service data from R

# Print links for notable sales

## Top sale

In [62]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-2201/home/187292011


## Cheapest sale

In [63]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Boca-Raton/161-Preston-D-33434/home/42251483


## Highest PSF

In [64]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Riviera-Beach/1940-Freeport-Ln-33404/home/42615201


## Lowest PSF

In [65]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Lake-Worth/2811-Garden-Dr-S-33461/unit-101/home/42283738


## Newest

In [66]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/West-Palm-Beach/2323-Lake-Dr-33404/unit-301/home/190633399


## Oldest

In [170]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Ocean-Ridge/5700-Old-Ocean-Blvd-33435/unit-R/home/42432027


## Time on Market Calculator

In [171]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 5) ## List (Earlier) date
date2 = datetime(2024, 8, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

68
